In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv("/home/sanju/Desktop/word2vec/labeledTrainData.tsv", header=0, \
                    delimiter="\t",quoting = 3)
test = pd.read_csv("/home/sanju/Desktop/word2vec/testData.tsv", header=0, \
                    delimiter="\t",quoting = 3)

In [4]:
unlabeled = pd.read_csv("/home/sanju/Desktop/word2vec/unlabeledTrainData.tsv", header=0, \
                    delimiter="\t",quoting = 3)

In [5]:
print test.head()
print train.head()
print unlabeled.head()

           id                                             review
0  "12311_10"  "Naturally in a film who's main themes are of ...
1    "8348_2"  "This movie is a disaster within a disaster fi...
2    "5828_4"  "All in all, this is a movie for kids. We saw ...
3    "7186_2"  "Afraid of the Dark left me with the impressio...
4   "12128_7"  "A very accurate depiction of small time mob l...
         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...
          id                                             review
0   "9999_0"  "Watching Time Chasers, it obvious that it was...
1  "45057_0"  "I saw this film about 2

In [6]:
all_train = train.append(unlabeled,ignore_index = True)
data = all_train.append(test,ignore_index = True)
print data.shape
print all_train.shape
print train.shape
print test.shape

(100000, 3)
(75000, 3)
(25000, 3)
(25000, 2)


In [7]:
data.head()

,id,review,sentiment
0,"""5814_8""","""With all this stuff going down at the moment ...",1.0
1,"""2381_9""","""\""The Classic War of the Worlds\"" by Timothy ...",1.0
2,"""7759_3""","""The film starts with a manager (Nicholas Bell...",0.0
3,"""3630_4""","""It must be assumed that those who praised thi...",0.0
4,"""9495_8""","""Superbly trashy and wondrously unpretentious ...",1.0


## Preprocessing

In [8]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

In [9]:
# combing all the above into one function
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  

In [10]:
clean_review = review_to_words( train["review"][0] )
print clean_review

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

/home/sanju/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/sanju/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [11]:
clean_reviews = []
for i in range(0,len(data.review)):
    clean_reviews.append(review_to_words(data.review[i]))

## Feature Extraction

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
data_features = vectorizer.fit_transform(clean_reviews)

In [13]:
train_features = data_features[:25000]
unlabel_features = data_features[25000:75000]
test_features = data_features[75000:]
print data_features.shape
print train_features.shape
print test_features.shape
print unlabel_features.shape

(100000, 5000)
(25000, 5000)
(25000, 5000)
(50000, 5000)


In [14]:
data_features = data_features.toarray()
train_features = train_features.toarray()
unlabel_features = unlabel_features.toarray()
test_features = test_features.toarray()
print len(vectorizer.vocabulary_)

5000


## Fit and Predict

In [69]:
from sklearn.ensemble import RandomForestClassifier
clf =  RandomForestClassifier(n_estimators = 95,oob_score = True, n_jobs = -1, random_state =50, max_features = "sqrt", min_samples_leaf = 5)

In [70]:
clf.fit(train_features,train['sentiment'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=95, n_jobs=-1, oob_score=True, random_state=50,
            verbose=0, warm_start=False)

In [71]:
#checking if overfitting
y_train_predict = clf.predict(train_features)
from sklearn.metrics import accuracy_score
print accuracy_score(y_train_predict,train['sentiment'])

0.932


In [75]:
y_unlabel_pred = clf.predict(unlabel_features)
y_test_pred = clf.predict(test_features)

## Output

In [73]:
#output = pd.read_csv("/home/sanju/Desktop/word2vec/testData.tsv", header=0, \
 #                   delimiter="\t")
#output = output.drop('review',axis = 1)

In [76]:
#output['sentiment'] = pd.Series(y_test_pred)   
#output.head()
#output.to_csv('submission3.csv',index = False)

In [77]:
unlabeled['sentiment'] = pd.Series(y_unlabel_pred)

In [78]:
new_train = train.append(unlabeled,ignore_index=True)
new_train_features = data_features[:75000]

In [79]:
clf2 =  RandomForestClassifier(n_estimators = 95,oob_score = True, n_jobs = -1, random_state =50, max_features = "sqrt", min_samples_leaf = 5)
clf2.fit(new_train_features, new_train['sentiment'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=95, n_jobs=-1, oob_score=True, random_state=50,
            verbose=0, warm_start=False)

In [83]:
new_y_test_pred = clf2.predict(test_features)

In [84]:
output2 = pd.read_csv("/home/sanju/Desktop/word2vec/testData.tsv", header=0, \
                   delimiter="\t")
output2 = output2.drop('review',axis = 1)
output2['sentiment'] = pd.Series(new_y_test_pred)
output2.to_csv('new_submission.csv',index = False)

In [82]:
new_y_train_predict = clf2.predict(new_train_features)
print accuracy_score(new_y_train_predict,new_train['sentiment'])

0.963106666667
